#ORION TEAM (16B)

NAMA ANGGOTA
1. Wahyu Rahmadani (SC61611)
2. Hildha Meutia K (SC61614)
3. Rafasya Azizah Ghassani (SC61617)
4. Tri Ayunia Patma Lubis (SC61618)
5. Berlian Angga Kusuma (SC61620)
6. M Rendy Apriansyah (SC61622)

In [ ]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords, words, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

Ada beberapa libraries yang kami pakai disini, scikit-learn, string, natural language toolkit, dan bs4 untuk scrap artikel via internet.

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

Disini kami juga mendownload beberapa package yang kami rasa diperlukan.

In [ ]:
def scrape(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  articles = []
  for article in soup.find_all('article'):
    text = article.get_text(separator=' ')
    articles.append(text)
  return articles

Diatas adalah fungsi untuk scrape data via url, memanfaatkan bs4 kita dapat secara otomatis mengambil data dari internet, selama situs yang dijadikan target berupa open-sources, maka tidak akan masalah.

Terkecuali website target memiliki privacy dan keamanan yang ketat.

In [ ]:
health_articles = scrape('https://www.who.int/news/item/26-04-2024-statement-on-the-antigen-composition-of-covid-19-vaccines')

sports_articles = scrape('https://economictimes.indiatimes.com/news/sports/ipl-2024-dc-vs-mi-pitch-arun-jaitley-stadium-playing-11-head-to-head-delhi-captials-mumbai-indians-dream-11-prediction-key-players-head-dc-vs-mi-today-match/articleshow/109640011.cms')

finance_articles = scrape('https://economictimes.indiatimes.com/markets/cryptocurrency/how-does-bitcoin-halving-impact-the-bitcoin-price/articleshow/109410406.cms')

Ada beberapa artikel yang kami ambil untuk dijadikan fondasi dataset, yaitu artikel terkait kesehatan, olahraga dan keuangan.

In [ ]:
print("Health Article:")
print(health_articles[0] if health_articles else "Data tidak ada.")

print("\nSports Article:")
print(sports_articles[0] if sports_articles else "Data tidak ada.")

print("\nFinance Article:")
print(finance_articles[0] if finance_articles else "Data tidak ada.")

Health Article:

 Key points SARS-CoV-2 continues to circulate and evolve with important genetic and antigenic evolution of the spike protein. The objective of an update to COVID-19 vaccine antigen composition is to enhance vaccine-induced immune responses to circulating SARS-CoV-2 variants. As the virus is expected to continue to evolve from JN.1, the TAG-CO-VAC advises the use of a  monovalent JN.1 lineage  as the antigen in future formulations of COVID-19 vaccines. In accordance with WHO SAGE policy, vaccination programmes should continue to use any of the WHO emergency-use listed or prequalified COVID-19 vaccines and vaccination should not be delayed in anticipation of access to vaccines with an updated composition. The WHO  Technical Advisory Group on COVID-19 Vaccine Composition   (TAG-CO-VAC) meets regularly to assess the impact of SARS-CoV-2 evolution on the performance of approved COVID-19 vaccines. This includes meeting in person approximately every six months to determine th

Berikut adalah hasil data scraping yang kami dapatkan menggunakan bs4. Diatas data belum diproses lebih lanjut.

Kami juga mengantisipasi jika data tidak ter-scrape menggunakan metode if, jika data tidak ter-append oleh fungsi maka tampilkan peringatan bahwa "Data tidak ada.". Dengan cara ini kita memiliki insight untuk mengambil url lain yang lebih terbuka.

In [ ]:
def preprocess(text):
  text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
  text = re.sub(r'\w\d\w', '', text)
  text = re.sub(r'[0-9]+', '', text)

  text = text.lower()

  # Tokenization
  words = word_tokenize(text)

  # Hapus stopwords
  stop_words = set(stopwords.words('english'))
  words = [word for word in words if word not in stop_words]

  # Lemmatization
  lemmatizer = WordNetLemmatizer()
  words = [lemmatizer.lemmatize(word) for word in words]

  return words

Setelah data collecting selesai, langkah selanjutnya adalah kita melakukan preproses terhadap data.

Ada beberapa tahapan dari yang telah kami riset, yaitu:

Tokenization - Stopwords - Lemmatization

Kami disini tidak melakukan proses stemming karena kami rasa dengan lematization sudah cukup.

In [ ]:
prepro_health = [preprocess(article) for article in health_articles]
prepro_sports = [preprocess(article) for article in sports_articles]
prepro_finance = [preprocess(article) for article in finance_articles]

In [ ]:
print("Prepro Health Data: ", prepro_health)
print("\nPrepro Sports Data: ", prepro_sports)
print("\nPrepro Finances Data: ", prepro_finance)

Prepro Health Data:  [['key', 'point', 'sarscov', 'continues', 'circulate', 'evolve', 'important', 'genetic', 'antigenic', 'evolution', 'spike', 'protein', 'objective', 'update', 'covi', 'vaccine', 'antigen', 'composition', 'enhance', 'vaccineinduced', 'immune', 'response', 'circulating', 'sarscov', 'variant', 'virus', 'expected', 'continue', 'evolve', 'jn', 'tagcovac', 'advises', 'use', 'monovalent', 'jn', 'lineage', 'antigen', 'future', 'formulation', 'covi', 'vaccine', 'accordance', 'sage', 'policy', 'vaccination', 'programme', 'continue', 'use', 'emergencyuse', 'listed', 'prequalified', 'covi', 'vaccine', 'vaccination', 'delayed', 'anticipation', 'access', 'vaccine', 'updated', 'composition', 'technical', 'advisory', 'group', 'covi', 'vaccine', 'composition', 'tagcovac', 'meet', 'regularly', 'ass', 'impact', 'sarscov', 'evolution', 'performance', 'approved', 'covi', 'vaccine', 'includes', 'meeting', 'person', 'approximately', 'every', 'six', 'month', 'determine', 'implication', 'sa

Berikut adalah data preprocessed. Terlihat bahwa data dipotong-potong per katanya.

Tahapan selanjutnya adalah untuk membuat BoW atau Bag of Words.

In [ ]:
# Bag of Words
health_text = [' '.join(article) for article in prepro_health]
sports_text = [' '.join(article) for article in prepro_sports]
finance_text = [' '.join(article) for article in prepro_finance]

# Satukan semua
bow = health_text + sports_text + finance_text

# Vectorization
vectorizer = CountVectorizer()

X_train = vectorizer.fit_transform(bow).toarray()

# Bagi BoW kedalam kelasnya
num_health_texts = len(health_text)
num_sports_texts = len(sports_text)
num_finance_texts = len(finance_text)

X_health = X_train[:num_health_texts]
X_sports = X_train[num_health_texts:num_health_texts + num_sports_texts]
X_finance = X_train[num_health_texts + num_sports_texts:]

Disini kita membuat 3 bag of words terpisah terlebih dahulu, yaitu menjadi health_text, sports_text, finance_text.

Setelah itu kita menyatukan semuanya menjadi variabel bow.

Setelah BoW terbuat kami melakukan proses vektorasi untuk melihat dan menentukan bobot kata, supaya data text bisa diproses oleh mesin nantinya. Karena bagaimanpun mesin hanya akan bisa memproses angka.

Vectorization selesai, lalu kami membagi BoW berdasarkan kelasnya, yaitu ada kelas health, sports dan finance.

In [ ]:
y_train = ['health', 'sports', 'finance']

# Satukan semua BoW menjadi satu dataset
X_train = np.concatenate((X_health, X_sports, X_finance))

# Proses training
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

Setelah data siap untuk ditraining, proses pemodelan pun dimulai, disini kami menggunakan Multinomial (Naive-Bayes).

In [ ]:
# Input test
input = "Cristiano Ronaldo came off the bench to earn Manchester United a hard-fought 2-1 victory at Everton in the Premier League on Sunday, taking his career goal tally to 700 in the process. Just as United did last weekend in their derby mauling at the hands of local rivals Manchester City, they again found themselves behind early on at Goodison Park after Alex Iwobi curled a sublime strike into the net from 20 metres."

# Preprocess input test
input_processed = ' '.join(preprocess(input))

input_bow = vectorizer.transform([input_processed]).toarray()

# Prediksi kelas
kelas = classifier.predict(input_bow)[0]
print("Hasil prediksi:", kelas)

Hasil prediksi: sports


Setelah itu seperti yang kita lihat diatas, kita melakukan test model terhadap input, dan input akan diprediksi termasuk kelas apakah dia.

Terlihat pada output bahwa input masuk kedalam kelas sports.